In [1]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

In [2]:
# api_key 저장
api_key = 'RGAPI-ca44b272-60ec-4589-8980-8f28e65ca212' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

#### ranker name puuid CSV 파일의 puuid 사용해서 모든 puuid 의 최근 20게임 game id 추출, 테이블화

##### 만들었던 csv 파일 불러와서 df 에 저장

In [3]:
df = pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/ranker_name_puuid.csv')

In [4]:
# 불러온 테이블 확인
df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,CHALLENGER,UF2IeZC87PAxxS0F7xg6VQNADum-YqoBLfYCOAb0LdmQL7WIfnII-3Cm1Q,kscv05,525,105,57,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ
1,CHALLENGER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,764,98,45,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
2,CHALLENGER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,586,103,65,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
3,CHALLENGER,0sgFw0oa2pxGV5rZR3Fi96VY8t6500V3t3TEso8XyyKmqHhQ4xURXo5oKA,lllIlIllIlIlIllI,646,76,19,M6OtnT2Ad4fhM9Sj1fh1odtr0p4oO6rqxc6qtW_lPlrkX8wfqUm-LMYHb_SRi56T8-X0npFDIZj9GQ
4,CHALLENGER,xszNgAg-6vRPAVyxqeu3L9oAeiFW8OnunPj6wbAbwAq_wVw,존잘인천피스트,529,101,78,7ITuO8wSX3-KnWipQVyAnnx7y75RPmOOU2TVy42-fSya7UXdwHfioyIG7jn6owP-tkUzrPfOU7Q4RQ


##### puuid 를 넣으면 최근 20게임의 game id 를 반환하는 함수 생성

In [5]:
def get_20_gameId(puuid):

    # puuid 을 넣어줄 url 저장
    puuid_url = "https://Asia.api.riotgames.com/tft/" + f'match/v1/matches/by-puuid/{puuid}/ids?count=20'

    # puuid 넣었을 때, 반환값 저장
    gameId_list = requests.get(puuid_url, headers = request_header).json()

    # puuid 추출
    return gameId_list

In [6]:
# 함수 작동 테스트
get_20_gameId('x5Aluxt-SEq_IUikKvH83sk0EkLXS7CP41bf66M_mBNdsiIUKYdavmHUPDKeWUBL6Ki5WwihBw2kQg')[:3]

['KR_7005813127', 'KR_7005746572', 'KR_7005704610']

##### get_20_gameId 함수를 활용하여 모든 puuid 의 최근 20 게임 gameId 를 gameId 컬럼을 만들어 추가

In [7]:
# 위에서 정의한 함수를 사용하여 모든 puuid에 대한 게임 ID 목록을 가져와서 저장할 빈 리스트 생성
gameId_list = []

# 각 행의 puuid를 함수에 넣어 게임 ID 목록을 가져와서 리스트에 추가
for puuid in df['puuid']:
    gameId_list.append(get_20_gameId(puuid))

# 새로운 DataFrame인 gameId_df 생성하여 gameId_list를 해당 컬럼에 추가
gameId_df = pd.DataFrame({'gameId': gameId_list})

# 기존 DataFrame(df)와 gameId_df를 합쳐서 새로운 DataFrame인 final_df를 생성
final_df = pd.concat([df, gameId_df], axis=1)

In [8]:
# 티어와 게임 아이디만 남기고 모두 제거
final_df = final_df[['tier', 'gameId']]

final_df.head(3)

,tier,gameId
0,CHALLENGER,"[KR_7005435650, KR_7005408175, KR_7005214080, KR_7005190828, KR_7005187292, KR_7005175450, KR_7005167344, KR_7005148547, KR_7005131948, KR_7005122693, KR_7005110331, KR_7005093596, KR_7005067255, KR_7005027397, KR_7004989030, KR_7004931557, KR_7004880295, KR_7004820438, KR_7004751585, KR_7004694291]"
1,CHALLENGER,"[KR_7006600541, KR_7006271996, KR_7006062589, KR_7005983989, KR_7005920322, KR_7005856311, KR_7005778536, KR_7005711341, KR_7005647676, KR_7005342411, KR_7005316238, KR_7005277758, KR_7005257964, KR_7005082056, KR_7005050597, KR_7005019336, KR_7004918979, KR_7004857407, KR_7004772195, KR_7004707761]"
2,CHALLENGER,"[KR_7006594228, KR_7006565737, KR_7006537939, KR_7006528195, KR_7005242847, KR_7005228320, KR_7005219085, KR_7005203110, KR_7005187292, KR_7005168757, KR_7005159721, KR_7005147991, KR_7005146365, KR_7005111937, KR_7005101344, KR_7005082056, KR_7005060446, KR_7005031593, KR_7004985790, KR_7004751585]"


##### 리스트로 되어있는 gameId 컬럼을 펼치고 중복값 제거하는 전처리 진행

In [9]:
# 리스트 형식으로 되어있는 gameId 컬럼 펼치기
final_df = final_df.explode('gameId')

final_df.head(3)

,tier,gameId
0,CHALLENGER,KR_7005435650
0,CHALLENGER,KR_7005408175
0,CHALLENGER,KR_7005214080


In [10]:
# gameId 컬럼 값이 중복된 데이터 제거
final_df = final_df.drop_duplicates(subset=['gameId'])

final_df

,tier,gameId
0,CHALLENGER,KR_7005435650
0,CHALLENGER,KR_7005408175
0,CHALLENGER,KR_7005214080
0,CHALLENGER,KR_7005190828
0,CHALLENGER,KR_7005187292
...,...,...
137,MASTER,KR_7005390721
137,MASTER,KR_7005363791
137,MASTER,KR_7005336302
137,MASTER,KR_7005149778


##### 전처리 완료한 테이블을 csv 파일로 저장

In [11]:
final_df.to_csv('tier_and_gameId.csv', index=False)

In [12]:
# 저장한 테이블 확인 (그랜드 마스터와 마스터가 섞여있는 큐는 그랜드 마스터 큐로 처리)
pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/tier_and_gameId.csv').head(193) ## 약 3분 1이 그마 큐

,tier,gameId
0,CHALLENGER,KR_7005435650
1,CHALLENGER,KR_7005408175
2,CHALLENGER,KR_7005214080
3,CHALLENGER,KR_7005190828
4,CHALLENGER,KR_7005187292
...,...,...
188,GRANDMASTER,KR_7005589355
189,GRANDMASTER,KR_7005409746
190,GRANDMASTER,KR_7005386147
191,GRANDMASTER,KR_7006345297
